<a href="https://colab.research.google.com/github/ChinaYiqun/recruitStudents/blob/main/faiss_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu
!pip install openai

In [6]:

import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
simTho = 0.5

def build_knowledge_rep(file):
    # 读取文件内容
    with open(file, 'r', encoding='utf-8') as f:
        texts = f.readlines()

    # 对每一行文本进行中文分词
    text_tokenized = [jieba.lcut(text.strip()) for text in texts]

    # 计算tf-idf
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform([' '.join(tokens) for tokens in text_tokenized])

    # 使用Faiss构建向量索引
    dim = tfidf_matrix.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(tfidf_matrix.toarray())

    return index, tfidf, text_tokenized,texts



def get_similarity_text(text, index, tfidf, text_tokenized,texts):
    # 对输入文本进行中文分词
    text_tokenized_input = jieba.lcut(text.strip())

    # 计算tf-idf
    tfidf_matrix_input = tfidf.transform([' '.join(text_tokenized_input)])

    # 使用向量索引查询相似向量
    sim, indices = index.search(tfidf_matrix_input.toarray(), k=1)

    if sim > simTho:
    # 返回相似向量所对应的文本内容
        similar_text = texts[indices[0][0]]

        return similar_text
    else:
        return ''


# 示例用法
index, tfidf, text_tokenized,texts = build_knowledge_rep('knowledge.txt')
similar_text = get_similarity_text('张爽', index, tfidf, text_tokenized,texts)
print(similar_text)

张爽，女，博士，副教授, 东北大学软件学院，辽宁省本科教学名师，软件学院教学指导委员会委员，软件学院星级教师，国家级一流本科课程《软件工程》负责人。近年来，作为项目负责人负责了江苏省镔鑫钢铁集团公司智能料场项目、广西盛隆冶金有限公司项目盛隆炼钢连铸一体化计划调度模型软件系统开发、冶金原料大数据分析模型软件系统、基于大数据分析技术的电力需求决策支持平台，均取得了非常好的应用效益，其中江苏省镔鑫钢铁集团公司智能料场为国内首个智能料场。作为子课题负责人承担了国家重点研发计划项目钢铁工业网络化协同生产智能管控平台关键技术研究子课题、和东北大学基础科研课题“基于岩体破裂大数据的金属矿山安全高效开采基础”。作为主要成员，参加了国家自然科学基金重点项目“机器学习驱动的知识平面创新和网络性能优化的研究”、中国工程院高端智库项目企业内部工业互联网网络系统发展研究、和辽宁省兴辽英才计划项目面向未来互联网的数据传输与安全防护技术。研究方向：人工智能、工业软件、信息技术应用创新、大数据+、工业互联网、物联网



In [21]:
messages_history = [
        {"role": "system", "content": "你扮演一个教师,根据角色内容进行对话,你的目标是作为这个教师招收优秀的研究生,说话方式可以搞笑一些,角色内容:"},
        ]


In [22]:
import openai
# 设置OpenAI的API秘钥
openai.api_key = '自己申请'


def chatgpt(input):
    # 发送请求给ChatGPT 3.5 API

    # max_tokens=100,   # 控制输出的长度
    # temperature=0.7,  # 控制生成文本的随机性。较低的值会更加保守，较高的值会更加随机。
    # n=1,              # 控制生成多少个候选答案
    # stop=None,        # 控制生成文本的停止条件
    similar_text = get_similarity_text(input, index, tfidf, text_tokenized,texts)
    messages_history[0]["content"] += similar_text
    messages_history.append({"role": "user", "content": input})
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages_history
    )

    # 解析API的响应并返回结果
    output = completion.choices[0].message.content
    messages_history.append({"role": "assistant", "content": output})
    return output



In [ ]:
while(True):
    q = input()
    r = chatgpt(q)
    print('->:'+r)

In [ ]:
messages_history